In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext magic_duckdb



In [ ]:
%%dql -t df

create table xyz as select * from range(10) x(b)
;
create table abc as select * from range(10) t(a)


In [ ]:
import sqlparse
import graphviz
import pathlib
#graphviz.backend.dot_command.DOT_BINARY = pathlib.Path("c:\\Program files\\graphviz\\bin\\dot")  # type: ignore # noqa


In [ ]:
import sqlparse
import graphviz
import pathlib
graphviz.backend.dot_command.DOT_BINARY = pathlib.Path("c:\\Program files\\graphviz\\bin\\dot")  # type: ignore # noqa

text="select abc.asd as dsef from abc, xyz"
from sqlparse.sql import TokenList, Identifier, Statement
from sqlparse.tokens import Whitespace
import re
import graphviz
from dataclasses import dataclass
from typing import Dict, List

dot = graphviz.Digraph()  # type: ignore # noqa

@dataclass
class Node:
    id: int
    name: str
    parent: "Node"
    properties: Dict

nodes: List[Node] = []
edges: List[Dict] = []


def get_node(name, parent):
    id = len(nodes)
    node = Node(id=id, name=name, parent=parent, properties={})
    nodes.append(node)
    return node

def get_props(o: object) -> Dict:
    # Do nothing for now, seems useless
    return {}

    result = {}
    
    for attr_name in dir(o):
        if attr_name.startswith("get_"):
            try:
                method = getattr(o, attr_name)
                result_key = attr_name[4:]  # Remove the "get_" prefix
                result_value = method()
                if result_value is not None:
                    result[result_key] = result_value
            except Exception as e:
                print(e) 
    print(result)
    return result

lasttoken=None
def process_node(tokens: sqlparse.sql.TokenList, parent: Node):
    global lasttoken
    for i, token in enumerate(tokens):       

        if token.ttype == Whitespace:
            continue
        name = token.value
        nodetype = type(token).__name__
        props = get_props(token)

        has_children = isinstance(token, TokenList) and token.is_group
        #if  has_children: 
        #    short_type = nodetype.replace("Identifier", "Id")
        node = get_node(f"{name}", parent)
        node.props=props

        if parent is not None:
            edges.append((parent.id, node.id))  # type: ignore

        if has_children:
            lasttoken = token

            process_node(token.tokens, node)

statements=sqlparse.parse(text)
for s in statements:
    node = get_node("Statement", None)

    process_node(s.tokens, node)

for node in nodes:
    dot.node(f"{node.id}", f"<{node.name}>", weight="99", shape="rectangle")

for e in edges:
    dot.edge(f"{e[0]}", f"{e[1]}", "weight=99", color="red")

dot

## TODO: Sizing Box vs Weighting Links